In [1]:
from pystac.extensions.eo import EOExtension as eo
import pystac_client
import planetary_computer
import json
import rasterio
from rasterio import windows
from rasterio import features
from rasterio import warp
from PIL import Image
from IPython.display import display
import numpy as np
from PIL import Image

In [2]:
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

In [3]:
# Load the GeoJSON data from the file
with open("dataset/train-Copy1.geojson", "r") as geojson_file:
    geojson_data = json.load(geojson_file)

# Iterate through the features
for feature in geojson_data["features"]:
    coordinates = feature["geometry"]["coordinates"]
    area_of_interest = {
        "type": "Polygon",
        "coordinates": coordinates,
    }
    
    time_of_interest = "2022-01-01/2022-12-30"
    
    search = catalog.search(
    collections=["sentinel-2-l2a"],
    intersects=area_of_interest,
    datetime=time_of_interest,
    query={"eo:cloud_cover": {"lt": 10}},)

    # Check how many items were returned
    items = search.item_collection()
    print(f"Returned {len(items)} Items")
    
    least_cloudy_item = min(items, key=lambda item: eo.ext(item).cloud_cover)

    print(
        f"Choosing {least_cloudy_item.id} from {least_cloudy_item.datetime.date()}"
        f" with {eo.ext(least_cloudy_item).cloud_cover}% cloud cover"
    )
    
    asset_href = least_cloudy_item.assets["visual"].href

    with rasterio.open(asset_href) as ds:
        aoi_bounds = features.bounds(area_of_interest)
        warped_aoi_bounds = warp.transform_bounds("epsg:4326", ds.crs, *aoi_bounds)
        aoi_window = windows.from_bounds(transform=ds.transform, *warped_aoi_bounds)
        band_data = ds.read(window=aoi_window)
    
    img = Image.fromarray(np.transpose(band_data, axes=[1, 2, 0]))
    w = img.size[0]
    h = img.size[1]
    aspect = w / h
    target_w = 800
    target_h = int(target_w / aspect)
    img = img.resize((target_w, target_h), Image.BILINEAR)
#     display(img)
    img.save(f"images_test_new/final_image_{feature['properties']['id']}.jpg", "JPEG")


Returned 19 Items
Choosing S2A_MSIL2A_20220401T165851_R069_T14QMG_20220402T074055 from 2022-04-01 with 1e-05% cloud cover
Returned 49 Items
Choosing S2B_MSIL2A_20220630T093039_R136_T34TGS_20220630T232604 from 2022-06-30 with 0.001051% cloud cover
Returned 19 Items
Choosing S2B_MSIL2A_20220428T130239_R095_T23KPQ_20220429T070446 from 2022-04-28 with 0.015456% cloud cover
Returned 1 Items
Choosing S2B_MSIL2A_20220112T022039_R003_T51PTS_20220112T211208 from 2022-01-12 with 5.427695% cloud cover
Returned 12 Items
Choosing S2B_MSIL2A_20220227T073849_R092_T37MBU_20220302T175301 from 2022-02-27 with 0.042714% cloud cover
Returned 28 Items
Choosing S2B_MSIL2A_20220620T093039_R136_T34TET_20220621T140717 from 2022-06-20 with 0.000435% cloud cover
Returned 9 Items
Choosing S2B_MSIL2A_20220207T160429_R097_T16PEU_20220221T092128 from 2022-02-07 with 0.022858% cloud cover
Returned 4 Items
Choosing S2A_MSIL2A_20220718T025541_R032_T48MYU_20220718T142042 from 2022-07-18 with 3.841336% cloud cover
Return